In [1]:
import os
import torch,torchvision
import torch.nn as nn


In [2]:
print(torch.__version__)

1.8.1+cu102


In [3]:
import _init_paths
import sys

In [4]:
sys.path

['/home/superorange5/Research/FedWCD/data/coco/PythonAPI',
 '/home/superorange5/Research/FedWCD/lib',
 '/home/superorange5/Research/FedWCD',
 '/home/superorange5/.conda/envs/py3/lib/python38.zip',
 '/home/superorange5/.conda/envs/py3/lib/python3.8',
 '/home/superorange5/.conda/envs/py3/lib/python3.8/lib-dynload',
 '',
 '/home/superorange5/.local/lib/python3.8/site-packages',
 '/home/superorange5/.conda/envs/py3/lib/python3.8/site-packages']

In [7]:
!pip install easydict

You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


In [8]:
from model.faster_rcnn.vgg16 import vgg16

ImportError: cannot import name '_C' from 'model' (/home/superorange5/Research/FedWCD/lib/model/__init__.py)

In [9]:
import numpy as np
KAIST_classes = np.asarray(['__background__','person','people','cyclist'])

fasterRCNN = vgg16(KAIST_classes, pretrained=False, class_agnostic='class_agnostic')


In [10]:
checkpoint  = torch.load('models/vgg16/KAIST/wkAVG_cd/faster_rcnn_KAIST_AVG_10.pth')

In [13]:
KAIST_classes = np.asarray(['__background__','person','people','cyclist'])

fasterRCNN = vgg16(KAIST_classes, pretrained=False, class_agnostic='class_agnostic')
fasterRCNN.create_architecture()
fasterRCNN.eval()

vgg16(
  (RCNN_rpn): _RPN(
    (RPN_Conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 24, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 48, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer()
    (RPN_anchor_target): _AnchorTargetLayer()
  )
  (RCNN_proposal_target): _ProposalTargetLayer()
  (RCNN_roi_pool): ROIPool(output_size=(7, 7), spatial_scale=0.0625)
  (RCNN_roi_align): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0)
  (RCNN_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), 

In [14]:
#checkpoint['model']['RCNN_bbox_pred.weight'] = checkpoint['model']['RCNN_bbox_pred.weight'][:4]
#checkpoint['model']['RCNN_bbox_pred.bias'] = checkpoint['model']['RCNN_bbox_pred.bias'][:4]

fasterRCNN.load_state_dict(checkpoint['model'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [15]:
fasterRCNN.to('cuda')

vgg16(
  (RCNN_rpn): _RPN(
    (RPN_Conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 24, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 48, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer()
    (RPN_anchor_target): _AnchorTargetLayer()
  )
  (RCNN_proposal_target): _ProposalTargetLayer()
  (RCNN_roi_pool): ROIPool(output_size=(7, 7), spatial_scale=0.0625)
  (RCNN_roi_align): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0)
  (RCNN_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), 

In [ ]:
 dynamic_axes={'input': [0, 1, 2, 3],
#                                 'input_1': [2, 3]
#                                }
#                   dynamic_axes={'input': {2: 'height', 3: 'width'}}
#                   dynamic_axes={'input': [0, 1, 2, 3]}

In [21]:
BATCH_SIZE = 16
model_name = "frcnn_vgg16_KAIST_AVG10.onnx"
#im_data, im_info, gt_boxes, num_boxes
dummy_input = (torch.randn( BATCH_SIZE,3,600, 750, device='cuda'),
               torch.randn( BATCH_SIZE,3, device='cuda'),
               torch.randn( BATCH_SIZE,20,5,  device='cuda'),
               torch.randn( BATCH_SIZE , device='cuda')
              )
dynamic_axes_input={'im_data': {0: 'batch', 2: 'height', 3: 'width'},  # shape(batch,3,600,750)
              'im_data': {0: 'batch'},  # shape(batch,3)                  
              'im_data': {0: 'batch'},  # shape(batch,20,5)   
              'im_data': {0: 'batch'},  # shape(batch)                  
                                    } 
output_name = ["rois", "cls_prob", "bbox_pred", "rpn_loss_cls", "rpn_loss_bbox", "RCNN_loss_cls", "RCNN_loss_bbox", "rois_label"]
#torch.onnx.export(model, dummy_input, model_name, verbose=True, keep_initializers_as_inputs=True)
torch.onnx.export(fasterRCNN, dummy_input, model_name, verbose=True, input_names=['im_data','im_info','gt_boxes','num_boxes'],output_names = output_name)

RuntimeError: Only tuples, lists and Variables supported as JIT inputs, but got int